# Задание 1. Градационные преобразования.

**Автор**: 22.М04, Винник Екатерина Петровна

**Задание**: реализовать градационные преобразования: 
   - [+] негатив 
   - [+] линейные 
   - [+] логарифмические
   - [+] степенные
   - [+] кусочно-линейные.

## Линейные градационные преобразования

### Кусочно-линейная функция 

In [1]:
import numpy as np


def pwl_transform(img, r1=.5, r2=.5, s1=0., s2=1.): # the values are normalized to the range [0, 1]
    
    # normalize the image
    img_norm = img / 255.
    
    # calculate the coefficients for the equation y = mx + b 
    k1 = s1 / r1
    k2 = (s2 - s1) / (r2 - r1)
    b2 = (r2*s1 - r1*s2) / (r2 - r1)
    k3 = (1 - s2) / (1 - r2)
    b3 = (s2 - r2) / (1 - r2)
    
    # transform the three intensity zones separately
    img_new_norm = img_norm
    img_1_idx = img_norm < r1
    img_temp = np.zeros_like(img_norm, dtype=np.float32)
    img_temp[img_norm >= r1] = 1.
    img_temp[img_norm >= r2] = 0.
    img_2_idx = img_temp == 1.
    img_3_idx = img_norm >= r2
    
    img_new_norm[img_1_idx] = k1 * img_norm[img_1_idx]
    img_new_norm[img_2_idx] = k2 * img_norm[img_2_idx] + b2
    img_new_norm[img_3_idx] = k3 * img_norm[img_3_idx] + b3
    
    img_new = img_new_norm * 255.
    
    return img_new

#### Пример применения 

In [2]:
import numpy as np
from PIL import Image

# From R. Gonzales, R. Woods "Digital Image Processing"
img_1 = Image.open('./images/eins.tif')

In [3]:
# represent the images as tensors
img_1_np = np.array(img_1)
print('Picture dimensions:')
print(img_1_np.shape)

Picture dimensions:
(600, 490)


In [4]:
# arbitrary values
img_1_np_pwl = pwl_transform(img_1_np, .35, .65, .15, .85)
img_1_pwl = Image.fromarray(img_1_np_pwl.astype(np.uint8))

In [5]:
img_1 = Image.fromarray(img_1_np.astype(np.uint8))
img_1_pwl.save('./eins_pwl.png')
img_1.save('./eins.png')

<p>
    <br>
    <img src="./eins.png" align="left"/> <img src="./eins_pwl.png" align="left"/>
    <br>
    <br>
    <div align="center">
        <em>Фотография Эйнштейна до и после кусочно-линейного преобразования</em>
    </div>
</p>

### Пороговая функция

In [6]:
import numpy as np


def trsh_transform(img, m):
    
    # use the pwl_transform function to obtain results
    img_new = np.zeros_like(img)
    img_new[img > m] = 255
    
    return img_new

#### Пример применения

In [7]:
# From R. Gonzales, R. Woods "Digital Image Processing"
img_3 = Image.open('./images/dollar.tif')

In [8]:
img_3_np = np.array(img_3)
print('Picture dimensions:')
print(img_3_np.shape)

Picture dimensions:
(500, 1192)


In [9]:
# using the threshold transform
img_3_mean = np.mean(img_3_np)
img_3_np_trsh = trsh_transform(img_3_np, img_3_mean)
img_3_trsh = Image.fromarray(img_3_np_trsh.astype(np.uint8))

In [10]:
img_3_trsh.save('./dollar_trsh.png')
img_3.save('./dollar.png')

<p>
    <br>
    <img src="./dollar.png" align="left" style="width:50%"/> <img src="./dollar_trsh.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em>Фотография доллара до и после преобразования с использованием пороговой функции</em>
    </div>
</p>

## Негатив

In [11]:
def neg_transform(img):
   
    img_new = 255 - img
    
    return img_new

#### Пример применения

In [12]:
# From R. Gonzales, R. Woods "Digital Image Processing"
img_21 = Image.open('./moon.tif')

In [13]:
# represent the images as tensors
img_21_np = np.array(img_21)

In [14]:
print('Picture dimensions:')
print(img_21_np.shape)

Picture dimensions:
(540, 466)


In [15]:
# negative of a mammogram
img_21_np_neg = neg_transform(img_21_np)
img_21_neg = Image.fromarray(img_21_np_neg.astype(np.uint8))

In [16]:
# save the results (and originals if necessary) as PNG
img_21 = Image.fromarray(img_21_np.astype(np.uint8))
img_21_neg.save('./moon_neg.png')
img_21.save('./moon.png')

<p>
    <br>
    <img src="./moon.png" align="left" style="width:50%"/> <img src="./moon_neg.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em>Луна до и после негатива</em>
    </div>
</p>

## Степенные градационные преобразования

In [17]:
import numpy as np


def power_transform(img, c=1., gamma=1.):
    
    img_norm = img / 255.
    img_norm_new = c * np.power(img_norm, gamma)
    if img_norm_new.max() > 1.:
        img_norm_new = img_norm_new / img_norm_new.max()
    img_new = img_norm_new * 255
        
    return img_new

#### Пример применения

In [18]:
# Loheland (materials for the course)
img_23 = Image.open('./field.jpg')
# Loheland (materials for the course)
img_24 = Image.open('./snow.jpg')

In [19]:
img_23_np = np.array(img_23)
img_24_np = np.array(img_24)
print('Picture dimensions:')

Picture dimensions:


In [20]:
print(img_23_np.shape)
print(img_24_np.shape)

(2736, 3648, 3)
(2736, 3648, 3)


In [21]:
# Gamma correction of a photo with a town on it
c = 1.
gamma = .8
img_23_np_pow = power_transform(img_23_np, c, gamma)
img_23_pow = Image.fromarray(img_23_np_pow.astype(np.uint8))

In [22]:
# Gamma correction of a photo with a ski slope
c = 1.
gamma = .7
img_24_np_pow = power_transform(img_24_np, c, gamma)
img_24_pow = Image.fromarray(img_24_np_pow.astype(np.uint8))

In [23]:
# save the results (and originals if necessary) as PNG
img_23_pow.save('./field_pow.png')
img_24_pow.save('./snow_pow.png')

<p>
    <br>
    <img src="./field.jpg" align="left" style="width:50%"/><img src="./field_pow.png" align="left" style="width:50%"/>
    <img src="./snow.jpg" align="left" style="width:50%"/><img src="./snow_pow.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em> Фотографии луга и снега до и после степенного образования при $\gamma = 0.8$ and $\gamma = 0.7$ соответственно</em>
    </div>
</p>

## Логарифмические градационные преобразования

In [24]:
import numpy as np


def log_transform(img, c=1.):
   
    img_norm = img / 255.
    img_norm_new = c * np.log2(img_norm + 1)
    if img_norm_new.max() > 1.:
        img_norm_new = img_norm_new / img_norm_new.max()
    img_new = img_norm_new * 255
    
    return img_new

#### Пример применения

In [25]:
# From R. Gonzales, R. Woods "Digital Image Processing"
img_22 = Image.open('./bone.tif')

In [26]:
img_22_np = np.array(img_22)
print('Picture dimensions:')
print(img_22_np.shape)

Picture dimensions:
(976, 746)


In [27]:
# log of a fractured spine
c = 1.
img_22_np_log = log_transform(img_22_np, c)
img_22_log = Image.fromarray(img_22_np_log.astype(np.uint8))

In [28]:
img_22 = Image.fromarray(img_22_np.astype(np.uint8))

In [29]:
img_22.save('./bone.png')
img_22_log.save('./bone_log.png')

<p>
    <br>
    <img src="./bone.png" align="left" style="width:50%"/><img src="./bone_log.png" align="left" style="width:50%"/>
    <br>
    <br>
    <div align="center">
        <em> Кость до и после логарифмического преобразования</em>
    </div>
</p>